In [1]:
from selenium import webdriver
from datetime import datetime
import pandas as pd
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as sc

In [2]:
def convert_strdate_date(str_):
    res_ = ' '.join(str_.split(' ')[1:])
    return datetime.strptime(res_,'%d %B %Y')


In [3]:
def lfc_is_home(str_):
    tmp = str_.split()
    if 'Liverpool' == tmp[0]:
        return True
    else :
        return False

#### fonction pour remplacer les tirets par 0

In [4]:
#### si liverpool joue a domicile il faut prendre les données a l'index 0 sinon prendre à l'index 2
#stocker dans le dataframe à la premiere ligne, les données recupérees les index du df seront les dates des matchs s
def replaceby0(tab):
    lst=[]
    for element in tab:
        if(element!=tab[3]):
            lst.append(element.replace("-","0"))
        else :
            lst.append(element)
    return lst

In [5]:
def insert_new_row(df,row):
    new_row = dict(zip(df.columns,row))
    new_df = df.append(new_row,ignore_index=True)
    return new_df

In [6]:
def recupData(tab,date):
    

    
    df = pd.DataFrame(columns=['Goals','Assists','Formation','Shots','Shots On Target','Saves','Penalties won','Penalties scored','Crosses','Fouls','Offsides','Corners','Yellow Card',
                       'Red Cards','Balls touched','Passes','Interceptions','Blocks','Catches','Tackles','Possession','Date'])
    
    if lfc_is_home(tab[0]) ==True:
        liste=[]
        for element in range(1,len(tab)):           
            if(element==4 or element ==6):
                liste.append(tab[element].split(' ')[0])
                liste.append(tab[element].split(' ')[1][tab[element].split(' ')[1].find("(")+1:tab[element].split(' ')[1].find(")")])
            
            else:
                liste.append(tab[element].split(' ')[0] )
    else :
        
        liste=[]
        for element in range(1,len(tab)):     
            
            if(element==4 ):
                liste.append(tab[element].split(' ')[5])
                liste.append(tab[element].split(' ')[6][tab[element].split(' ')[6].find("(")+1:tab[element].split(' ')[6].find(")")])
            elif(element==6):
                liste.append(tab[element].split(' ')[4])
                liste.append(tab[6].split(' ')[5][tab[6].split(' ')[5].find("(")+1:tab[6].split(' ')[5].find(")")])
            elif(element==11 or element==12):
                liste.append(tab[element].split(' ')[3])
                
            else :
                liste.append(tab[element].split(' ')[2] )   
    liste.append(date)
    return insert_new_row(df,liste)

In [21]:
driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')

In [25]:
driver.get('https://www.goalzz.com/main.aspx?region=-7&team=11')

In [26]:
lst_btn = driver.find_elements_by_xpath('.//button[@class="btnDetails"]')

In [27]:
for i in range(len(lst_btn) - 300,len(lst_btn) - 500,-1):
    lst_btn[i].click()

In [28]:
lst_btnVert=driver.find_elements_by_xpath('.//a[@class="hint--bottom hint--success" and @data-hint="Match details"]')

In [29]:
df_final = pd.DataFrame(columns=['Goals','Assists','Formation','Shots','Shots On Target','Saves','Penalties won','Penalties scored','Crosses','Fouls','Offsides','Corners','Yellow Card',
                       'Red Cards','Balls touched','Passes','Interceptions','Blocks','Catches','Tackles','Possession','Date'])

In [30]:
i = 0
for btn in lst_btnVert :
    i += 1
    print(f'Match : {i}')
    ActionChains(driver).context_click(btn).key_down(Keys.CONTROL).click().perform()
    driver.switch_to_window(driver.window_handles[-1])
    waiter = WebDriverWait(driver,timeout=10)
    try :
        tab_ele = waiter.until(sc.presence_of_element_located((By.XPATH,'.//td[@class="empty" and @valign="top"]')))
        tab = tab_ele.text
        tab=tab.split('\n')
        tab=replaceby0(tab)
        tab=tab[0:20]
        try : 
            date=driver.find_element_by_xpath('.//td[@class="match_date"]').text
            df_tmp = recupData(tab,date)
            df_final = df_final.append(df_tmp)
        except ValueError:
            print("erreur table ..")
    except IndexError:
        print('Match sans détails')
    driver.switch_to_window(driver.window_handles[0])

Match : 1


C:\Users\ibrah\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use driver.switch_to.window instead
  
C:\Users\ibrah\anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: use driver.switch_to.window instead


Match : 2
Match : 3
Match : 4
Match : 5
Match : 6
Match : 7
Match : 8
Match : 9
Match : 10
Match : 11
Match : 12
Match : 13
Match : 14
Match : 15
Match : 16
Match : 17
Match : 18
Match : 19
Match : 20
Match : 21
Match : 22
Match : 23
Match : 24
Match : 25
Match : 26
Match : 27
Match : 28
Match : 29
Match : 30
Match : 31
Match : 32
Match : 33
Match : 34
Match : 35
Match : 36
Match : 37
Match : 38
Match : 39
Match : 40
Match : 41
Match : 42
Match : 43
Match : 44
Match : 45
Match : 46
Match : 47
Match : 48
Match : 49
Match : 50
Match : 51
Match : 52
Match : 53
Match : 54
Match : 55
Match : 56
Match : 57
Match : 58
Match : 59
Match : 60
Match : 61
Match : 62
Match : 63
Match : 64
Match : 65
Match : 66
Match : 67
Match : 68
Match : 69
Match : 70
Match : 71
Match : 72
Match : 73
Match : 74
Match : 75
Match : 76
Match : 77
Match : 78
Match : 79
Match : 80
Match : 81
Match : 82
Match : 83
Match : 84
Match : 85
Match : 86
Match : 87
Match : 88
Match : 89
Match : 90
Match : 91
Match : 92
Match :

In [31]:
df_final['Date'] = df_final['Date'].apply(convert_strdate_date)

In [32]:
df_final = df_final.reset_index(drop=True)

In [33]:
df_final

,Goals,Assists,Formation,Shots,Shots On Target,Saves,Penalties won,Penalties scored,Crosses,Fouls,...,Yellow Card,Red Cards,Balls touched,Passes,Interceptions,Blocks,Catches,Tackles,Possession,Date
0,3,2,4-2-3-1,14,5,1,0,0,12,4,...,0,0,786,489,47,4,0,3,61%,2012-04-28
1,0,0,4-4-2,17,4,4,0,0,36,12,...,0,0,753,461,43,13,0,16,56%,2012-05-01
2,4,2,4-4-1-1,24,6,2,1,0,12,17,...,2,0,624,351,56,3,0,13,49%,2012-05-08
3,0,0,4-4-2,13,6,4,0,0,19,5,...,1,0,335,164,19,7,1,1,33%,2012-05-13
4,0,0,4-3-3,16,2,4,0,0,17,12,...,4,1,635,378,22,13,3,13,62%,2012-08-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,1,0,4-4-2,8,4,0,0,0,9,11,...,2,0,642,333,23,8,0,33,45%,2015-09-12
129,1,1,3-5-2,22,7,1,0,0,18,4,...,1,0,776,469,32,3,0,8,56%,2015-09-20
130,3,3,3-4-1-2,21,12,2,0,0,23,8,...,0,0,761,489,12,3,1,25,58%,2015-09-26
131,1,1,3-4-1-2,13,4,3,0,0,18,19,...,3,0,620,303,33,7,0,42,51%,2015-10-04


In [34]:
df_final.to_csv('details_match_goalzz_500.csv',index=False)

# Concate all csv

In [35]:
df_1 = pd.read_csv('details_match_goalzz_100.csv')

In [36]:
df_2 = pd.read_csv('details_match_goalzz_200.csv')

In [37]:
df_3 = pd.read_csv('details_match_goalzz_300.csv')

In [38]:
df_4 = pd.read_csv('details_match_goalzz_500.csv')

In [39]:
lst_df = [df_4,df_3,df_2,df_1]

In [48]:
df_final = pd.DataFrame(columns=['Goals','Assists','Formation','Shots','Shots On Target','Saves','Penalties won','Penalties scored','Crosses','Fouls','Offsides','Corners','Yellow Card',
                       'Red Cards','Balls touched','Passes','Interceptions','Blocks','Catches','Tackles','Possession','Date'])

In [49]:
for df in lst_df : 
    df_final = df_final.append(df)

In [50]:
df_final = df_final.reset_index(drop=True)

In [53]:
df_final.to_csv('details_match_goalzz_from_2012.csv',index=False)

# Merging matchs and details 

In [54]:
df_match = pd.read_csv('goalzz_archive_from_2012_v2.csv')

In [55]:
df_match

,date_match,contests,home_team,home_score,away_score,away_team,liverpool_score,at_anfield,season,rest_days
0,2012-08-18,English Premier League,West Bromwich Albion FC,3,0,Liverpool,D,False,2012 - 2013,6
1,2012-08-23,UEFA Europa League,Heart of Midlothian FC,0,1,Liverpool,V,False,2012 - 2013,5
2,2012-08-26,English Premier League,Liverpool,2,2,Manchester City,N,True,2012 - 2013,3
3,2012-08-30,UEFA Europa League,Liverpool,1,1,Heart of Midlothian FC,N,True,2012 - 2013,4
4,2012-09-02,English Premier League,Liverpool,0,2,Arsenal FC,D,True,2012 - 2013,3
...,...,...,...,...,...,...,...,...,...,...
470,2020-07-08,English Premier League,Brighton & Hove Albion,1,3,Liverpool,V,False,2019 - 2020,3
471,2020-07-11,English Premier League,Liverpool,1,1,Burnley FC,N,True,2019 - 2020,3
472,2020-07-15,English Premier League,Arsenal FC,2,1,Liverpool,D,False,2019 - 2020,4
473,2020-07-22,English Premier League,Liverpool,5,3,Chelsea,V,True,2019 - 2020,7


In [57]:
md  = pd.merge(df_match,df_final,left_on='date_match',right_on='Date',how='left')

In [75]:
md

,date_match,contests,home_team,home_score,away_score,away_team,liverpool_score,at_anfield,season,rest_days,...,Yellow Card,Red Cards,Balls touched,Passes,Interceptions,Blocks,Catches,Tackles,Possession,Date
0,2012-08-18,English Premier League,West Bromwich Albion FC,3,0,Liverpool,D,False,2012 - 2013,6,...,4,1,635,378,22,13,3,13,62%,2012-08-18
1,2012-08-23,UEFA Europa League,Heart of Midlothian FC,0,1,Liverpool,V,False,2012 - 2013,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-08-26,English Premier League,Liverpool,2,2,Manchester City,N,True,2012 - 2013,3,...,1,0,511,268,38,7,1,3,44%,2012-08-26
3,2012-08-30,UEFA Europa League,Liverpool,1,1,Heart of Midlothian FC,N,True,2012 - 2013,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-09-02,English Premier League,Liverpool,0,2,Arsenal FC,D,True,2012 - 2013,3,...,2,0,732,439,25,5,0,21,56%,2012-09-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,2020-07-08,English Premier League,Brighton & Hove Albion,1,3,Liverpool,V,False,2019 - 2020,3,...,4,0,1371,431,3,17,3,8,53%,2020-07-08
472,2020-07-11,English Premier League,Liverpool,1,1,Burnley FC,N,True,2019 - 2020,3,...,1,0,1968,600,6,12,5,5,66%,2020-07-11
473,2020-07-15,English Premier League,Arsenal FC,2,1,Liverpool,D,False,2019 - 2020,4,...,1,0,1706,558,6,15,5,7,64%,2020-07-15
474,2020-07-22,English Premier League,Liverpool,5,3,Chelsea,V,True,2019 - 2020,7,...,1,0,1356,484,2,10,2,7,49%,2020-07-22


In [76]:
md_non_null

,date_match,contests,home_team,home_score,away_score,away_team,liverpool_score,at_anfield,season,rest_days,...,Yellow Card,Red Cards,Balls touched,Passes,Interceptions,Blocks,Catches,Tackles,Possession,Date
0,2012-08-18,English Premier League,West Bromwich Albion FC,3,0,Liverpool,D,False,2012 - 2013,6,...,4,1,635,378,22,13,3,13,62%,2012-08-18
2,2012-08-26,English Premier League,Liverpool,2,2,Manchester City,N,True,2012 - 2013,3,...,1,0,511,268,38,7,1,3,44%,2012-08-26
4,2012-09-02,English Premier League,Liverpool,0,2,Arsenal FC,D,True,2012 - 2013,3,...,2,0,732,439,25,5,0,21,56%,2012-09-02
5,2012-09-15,English Premier League,AFC Sunderland,1,1,Liverpool,N,False,2012 - 2013,13,...,2,0,858,525,70,4,0,11,65%,2012-09-15
7,2012-09-23,English Premier League,Liverpool,1,2,Manchester United,D,True,2012 - 2013,3,...,1,1,541,299,14,1,1,11,54%,2012-09-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,2020-07-08,English Premier League,Brighton & Hove Albion,1,3,Liverpool,V,False,2019 - 2020,3,...,4,0,1371,431,3,17,3,8,53%,2020-07-08
472,2020-07-11,English Premier League,Liverpool,1,1,Burnley FC,N,True,2019 - 2020,3,...,1,0,1968,600,6,12,5,5,66%,2020-07-11
473,2020-07-15,English Premier League,Arsenal FC,2,1,Liverpool,D,False,2019 - 2020,4,...,1,0,1706,558,6,15,5,7,64%,2020-07-15
474,2020-07-22,English Premier League,Liverpool,5,3,Chelsea,V,True,2019 - 2020,7,...,1,0,1356,484,2,10,2,7,49%,2020-07-22


In [60]:
len(md[md['Yellow Card'].notnull()]) / len(md)

0.7226890756302521

In [67]:
md_non_null = md[md['Yellow Card'].notnull()]

In [71]:
md.groupby(['contests']).count()['date_match']

contests
Audi Cup                          1
Borussia Dortmund Pre-Season      1
England FA Cup                   26
English League Cup               21
English Premier League          305
FIFA Club World Championship      2
Friendlies - Clubs                1
International Champions Cup       9
Liverpool Pre-Season             40
Premier League Asia Trophy        2
UEFA Champions League            42
UEFA Europa League               26
Name: date_match, dtype: int64

In [72]:
md_non_null.groupby(['contests']).count()['date_match']

contests
English Premier League    302
UEFA Champions League      42
Name: date_match, dtype: int64

In [77]:
md_non_null.to_csv('all_details_match_goalzz.csv',index=False)